In [ ]:
---
title: "Credit Card Fraud Detection using RandomForest"
author: "Gabriel Preda"
date: "November 12, 2017"
output:
  html_document:
    number_sections: true
    toc: true
    fig_width: 8
    fig_height: 6
    theme: cosmo
    highlight: tango
    code_folding: show
---


```{r setup, include=FALSE}
library(randomForest)
library(caret)
library(gridExtra)
library(grid)
library(ggplot2)
library(lattice)
library(corrplot)
library(pROC)
library(knitr)
library(kableExtra)
library(formattable)
library(dplyr)
options(knitr.table.format = "html") 
```

# Introduction

This kernel uses and modifies code from 
[https://www.r-bloggers.com/illustrated-guide-to-roc-and-auc/](https://www.r-bloggers.com/illustrated-guide-to-roc-and-auc/).


We input the data.
```{r read_data}
raw.data <- read.csv("../input/creditcard.csv")
```

# Utility functions


We will use a function for calculation of ROC and AUC.


```{r utils_functions}
#calculate ROC (https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
calculate_roc <- function(verset, cost_of_fp, cost_of_fn, n=100) {
  
  tp <- function(verset, threshold) {
    sum(verset$predicted >= threshold & verset$Class == 1)
  }
  
  fp <- function(verset, threshold) {
    sum(verset$predicted >= threshold & verset$Class == 0)
  }
  
  tn <- function(verset, threshold) {
    sum(verset$predicted < threshold & verset$Class == 0)
  }
  
  fn <- function(verset, threshold) {
    sum(verset$predicted < threshold & verset$Class == 1)
  }
  
  tpr <- function(verset, threshold) {
    sum(verset$predicted >= threshold & verset$Class == 1) / sum(verset$Class == 1)
  }
  
  fpr <- function(verset, threshold) {
    sum(verset$predicted >= threshold & verset$Class == 0) / sum(verset$Class == 0)
  }
  
  cost <- function(verset, threshold, cost_of_fp, cost_of_fn) {
    sum(verset$predicted >= threshold & verset$Class == 0) * cost_of_fp + 
      sum(verset$predicted < threshold & verset$Class == 1) * cost_of_fn
  }
  fpr <- function(verset, threshold) {
    sum(verset$predicted >= threshold & verset$Class == 0) / sum(verset$Class == 0)
  }
  
  threshold_round <- function(value, threshold)
  {
    return (as.integer(!(value < threshold)))
  }
  #calculate AUC (https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve)
  auc_ <- function(verset, threshold) {
    auc(verset$Class, threshold_round(verset$predicted,threshold))
  }
  
  roc <- data.frame(threshold = seq(0,1,length.out=n), tpr=NA, fpr=NA)
  roc$tp <- sapply(roc$threshold, function(th) tp(verset, th))
  roc$fp <- sapply(roc$threshold, function(th) fp(verset, th))
  roc$tn <- sapply(roc$threshold, function(th) tn(verset, th))
  roc$fn <- sapply(roc$threshold, function(th) fn(verset, th))
  roc$tpr <- sapply(roc$threshold, function(th) tpr(verset, th))
  roc$fpr <- sapply(roc$threshold, function(th) fpr(verset, th))
  roc$cost <- sapply(roc$threshold, function(th) cost(verset, th, cost_of_fp, cost_of_fn))
  roc$auc <-  sapply(roc$threshold, function(th) auc_(verset, th))
  
  return(roc)
}
```

The following functions is for graphical representation of ROC, AUC and cost function.

```{r plot_roc_auc_cost_functions}
plot_roc <- function(roc, threshold, cost_of_fp, cost_of_fn) {
  library(gridExtra)
  
  norm_vec <- function(v) (v - min(v))/diff(range(v))
  
  idx_threshold = which.min(abs(roc$threshold-threshold))
  
  col_ramp <- colorRampPalette(c("green","orange","red","black"))(100)
  col_by_cost <- col_ramp[ceiling(norm_vec(roc$cost)*99)+1]
  p_roc <- ggplot(roc, aes(fpr,tpr)) + 
    geom_line(color=rgb(0,0,1,alpha=0.3)) +
    geom_point(color=col_by_cost, size=2, alpha=0.5) +
    labs(title = sprintf("ROC")) + xlab("FPR") + ylab("TPR") +
    geom_hline(yintercept=roc[idx_threshold,"tpr"], alpha=0.5, linetype="dashed") +
    geom_vline(xintercept=roc[idx_threshold,"fpr"], alpha=0.5, linetype="dashed")
  
  p_auc <- ggplot(roc, aes(threshold, auc)) +
    geom_line(color=rgb(0,0,1,alpha=0.3)) +
    geom_point(color=col_by_cost, size=2, alpha=0.5) +
    labs(title = sprintf("AUC")) +
    geom_vline(xintercept=threshold, alpha=0.5, linetype="dashed")
  
  p_cost <- ggplot(roc, aes(threshold, cost)) +
    geom_line(color=rgb(0,0,1,alpha=0.3)) +
    geom_point(color=col_by_cost, size=2, alpha=0.5) +
    labs(title = sprintf("cost function")) +
    geom_vline(xintercept=threshold, alpha=0.5, linetype="dashed")
  
  sub_title <- sprintf("threshold at %.2f - cost of FP = %d, cost of FN = %d", threshold, cost_of_fp, cost_of_fn)
  # 
  grid.arrange(p_roc, p_auc, p_cost, ncol=2,sub=textGrob(sub_title, gp=gpar(cex=1), just="bottom"))
}
```

The follwing function is for showing a confusion matrix.

```{r show_confusion_matrix}
plot_confusion_matrix <- function(verset, sSubtitle) {
    tst <- data.frame(round(verset$predicted,0), verset$Class)
    opts <-  c("Predicted", "True")
    names(tst) <- opts
    cf <- plyr::count(tst)
    cf[opts][cf[opts]==0] <- "Not Fraud"
    cf[opts][cf[opts]==1] <- "Fraud"
    
    ggplot(data =  cf, mapping = aes(x = True, y = Predicted)) +
      labs(title = "Confusion matrix", subtitle = sSubtitle) +
      geom_tile(aes(fill = freq), colour = "grey") +
      geom_text(aes(label = sprintf("%1.0f", freq)), vjust = 1) +
      scale_fill_gradient(low = "lightblue", high = "blue") +
      theme_bw() + theme(legend.position = "none")
  
}


```

# Explore the data

Let's glimpse the data:

```{r glimpse_data}
head(raw.data,10) %>%
  kable( "html", escape=F, align="c") %>%
   kable_styling(bootstrap_options = "striped", full_width = F, position = "center")
``` 

There are totally 31 columns in the data. One column, `Class` is the target value; it is a binary value, can
have either `0` (not fraud) or `1` (fraud) value. Another two columns have clear meaning: `Amount` is the
amount of the transaction; `Time` is the time of the transaction. The rest of the features (28), anonymized, are
named from `V1` to `V28`.

# Correlations

We represent the Pearson correlation for the data.

```{r correlation}
correlations <- cor(raw.data,method="pearson")
corrplot(correlations, number.cex = .9, method = "circle", type = "full", tl.cex=0.8,tl.col = "black")
```
We can observe that most of the data features are not correlated. This is because before publishing,
most of the features were presented to a Principal Component Analysis (PCA) algorithm.
The features `V1` to `V28` are most probably the Principal Components resulted after propagating
the real features through PCA. We do not know if the numbering of the features reflects the importance
of the Principal Components. This information might be checked partially using the Variable Importance
from Random Forest.

# Model

After we split the data in a training and test set, we create the RF model using the training set.

```{r model}
nrows <- nrow(raw.data)
set.seed(314)
indexT <- sample(1:nrow(raw.data), 0.7 * nrows)

#separate train and validation set
trainset = raw.data[indexT,]
verset =   raw.data[-indexT,]

n <- names(trainset)
rf.form <- as.formula(paste("Class ~", paste(n[!n %in% "Class"], collapse = " + ")))

trainset.rf <- randomForest(rf.form,trainset,ntree=100,importance=T)
```

For the trained model, let's see visualize the variable importance.

```{r variable_importance}
varimp <- data.frame(trainset.rf$importance)

  vi1 <- ggplot(varimp, aes(x=reorder(rownames(varimp),IncNodePurity), y=IncNodePurity)) +
  geom_bar(stat="identity", fill="tomato", colour="black") +
  coord_flip() + theme_bw(base_size = 8) +
  labs(title="Prediction using RandomForest with 100 trees", subtitle="Variable importance (IncNodePurity)", x="Variable", y="Variable importance (IncNodePurity)")
  
  vi2 <- ggplot(varimp, aes(x=reorder(rownames(varimp),X.IncMSE), y=X.IncMSE)) +
  geom_bar(stat="identity", fill="lightblue", colour="black") +
  coord_flip() + theme_bw(base_size = 8) +
  labs(title="Prediction using RandomForest with 100 trees", subtitle="Variable importance (%IncMSE)", x="Variable", y="Variable importance (%IncMSE)")

grid.arrange(vi1, vi2, ncol=2)
```

# Prediction

Let's use the trained model for prediction of the Fraud/Not Fraud Class for the test set.

```{r verif}
verset$predicted <- predict(trainset.rf ,verset)
```

For the threshold at 0.5, let's represent the Confusion matrix.

```{r fig.width=4, fig.height=4, confusion_matrix_}
plot_confusion_matrix(verset, "Random Forest with 100 trees")
```

For such a problem, where the number of TP is very small in comparison with the number of TN, the Confusion Matrix is less useful, since it is important to use a metric that include evaluation of FP and FN as well. It is important to minimize as much as possible the number of FN (Predicted: Not Fraud and True: Fraud) since their cost could be very large. Tipically AUC is used for such cases.

Let's calculate the TP, FP, TN, FN, ROC, AUC and cost for threshold with values
between 0 and 1 (100 values equaly distributed) and cost 1 for TN and 10 for FN.

```{r comp_roc_auc_cost_}
roc <- calculate_roc(verset, 1, 10, n = 100)

mincost <- min(roc$cost)
roc %>%
 mutate(
   auc = ifelse(cost == mincost,
                  cell_spec(sprintf("%.5f", auc), "html", color = "green", background = "lightblue", bold = T),
                  cell_spec(sprintf("%.5f", auc), "html", color = "black", bold = F))
  ) %>%
  kable( "html", escape=F, align="c") %>%
   kable_styling(bootstrap_options = "striped", full_width = F, position = "center") %>%
   scroll_box(height = "600px")
```


Let's plot the ROC, AUC and cost functions for a ref. threshold of 0.3.

```{r plot_roc_auc_cost_}
threshold = 0.3
plot_roc(roc, threshold, 1, 10)
```

# Conclusions

The calculated accuracy is not very relevant in the conditions where there is a very large unbalance between
the number of `fraud` and `non-fraud` events in the dataset. In such cases, we can see a very large accuracy.
More relevant is the value of ROC-AUC (Area Under Curve for the Receiver Operator Characteristic). The value
obtained (0.93) is relativelly good, considering that we did not performed any tunning, working with default
RF algorithm parameters.
